In [19]:
import numpy as np
import cv2

In [23]:
search_image = cv2.imread('search.png')
template_image = cv2.imread('template.png')

search_lab = cv2.cvtColor(search_image, cv2.COLOR_BGR2Lab)
template_lab = cv2.cvtColor(template_image, cv2.COLOR_BGR2Lab)


In [25]:
template_height, template_width = template_image.shape[:2]
# Calculating half_width and half_height to ensure template is placed entirely within search image
half_width = template_width // 2 
half_height = template_height // 2

# print(half_width)
# print(half_height)

In [26]:
scores = []
positions = []

# Iterating over the search image
for y in range(half_height, search_image.shape[0] - half_height):
    for x in range(half_width, search_image.shape[1] - half_width):
        # Extracting region of interest from search image and resizing it to template size
        roi = search_lab[y - half_height:y + half_height, x - half_width:x + half_width]
        roi_resized = cv2.resize(roi, (template_width, template_height))
        
        # Normalize template and resized region of interest
        normalized_template = (template_lab - np.mean(template_lab)) / np.std(template_lab)
        normalized_roi = (roi_resized - np.mean(roi_resized)) / np.std(roi_resized)

        ncc_metric = np.sum(normalized_template * normalized_roi) # Computing the color-based NCC metric

        scores.append(ncc_metric) # Storing score and position
        positions.append((x, y))

In [30]:
# Sorting the socres and positions in descending order
sorted_indices = np.argsort(scores)[::-1]
sorted_scores = np.array(scores)[sorted_indices]
sorted_positions = np.array(positions)[sorted_indices]

# Define the indices for specific matches to visualize
indices_to_visualize = [0, 1, 4, 9, 99, 499]

# Display patches for specific matches
for i in indices_to_visualize:
    x, y = sorted_positions[i]
    patch = search_image[y - half_height:y + half_height, x - half_width:x + half_width]
    cv2.imwrite(f'match_{i+1}.png', patch)
    cv2.imshow(f'Patch {i+1}', patch)

cv2.waitKey(0)
cv2.destroyAllWindows()